In [6]:
import os
# webdriver is frame work which allows us to execute test on browser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
# bs4 for parsing webpages
from bs4 import BeautifulSoup as soup
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

In [7]:
# selenium browser setup
def setup_browser():
    chrome_options = webdriver.ChromeOptions()
    # this is a flag which prevents selenium driven webdriver from being detected
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')
    # returns the current working directory
    prefs = {"download.default_directory" : os.getcwd()}
    chrome_options.add_experimental_option("prefs",prefs)
    browser = webdriver.Chrome('/Users/hemant/Downloads/chromedriver', options=chrome_options)
    browser.maximize_window()
    return browser

In [8]:
def get_stock_data(stock_symbol):
    # broswer setup and opening url
    browser = setup_browser()
    url = 'https://www.nseindia.com/'
    browser.get(url)
    # waiting for page load
    time.sleep(10)
    # searching stock by its symbol
    search = browser.find_element_by_xpath('//*[@id="header-search-input"]')
    search.send_keys(stock_symbol)
    search.send_keys(Keys.ENTER)
    time.sleep(5)
    search_result = browser.find_element_by_xpath('//*[@id="searchListing"]/div[1]/div/a')
    search_result.click()
    time.sleep(5)
    # switching to next tab as search result will open in seperate tab
    browser.switch_to.window(browser.window_handles[1])
    time.sleep(5)
    ###### NOT WORKING WITHOUT SCROLL
    ###### Page scroll down such that content will be on screen
    #browser.execute_script("window.scrollBy(0,800)","")
    ###### SOMETHING I DONT KNOW
    # getting historical data by clicking on historical data button
    historical_data_page = browser.find_elements_by_link_text('Historical Data')
    historical_data_page[0].click()
    time.sleep(2)
    # setting range to 1 week
    select_range = browser.find_element_by_xpath('//*[@id="historical-trade"]/section/div/div[1]/div/div[1]/ul/li[6]/a')
    select_range.click()
    # applying filter by clicking onto filter button after setting range
    filter_button = browser.find_element_by_xpath('//*[@id="equity-historical-Date-filter"]/div[3]/button')
    filter_button.click()
    time.sleep(10)
    # table content from historical data
    # table columns
    table_content = browser.find_element_by_xpath('//*[@id="equityHistoricalTable"]/thead')
    table_columns_html = soup(table_content.get_attribute('innerHTML'),'html.parser')
    column_names = []
    for column_name in table_columns_html.findAll('th'):
        column_names.append(column_name.text) 
    # table data
    table_stock_data = browser.find_element_by_xpath('//*[@id="equityHistoricalTable"]/tbody')
    table_stock_data = soup(table_stock_data.get_attribute('innerHTML'),'html.parser')
    stock_data = []
    for column_data in table_stock_data.findAll('td'):
        stock_data.append(column_data.text)
    browser.quit()
    # return column names from table and content of table 
    return column_names, stock_data

In [62]:
# setting up skip list for storing data
class Node(object):
    '''
    Class to implement node
    '''

    def __init__(self, key, level):
        self.key = key

        # list to hold references to node of different level
        self.forward = [None] * (level + 1)


class SkipList(object):
    '''
    Class for Skip list
    '''

    def __init__(self, max_lvl, P):
        # Maximum level for this skip list
        self.MAXLVL = max_lvl

        # P is the fraction of the nodes with level
        # i references also having level i+1 references
        self.P = P

        # create header node and initialize key to -1
        self.header = self.createNode(self.MAXLVL, -1)

        # current level of skip list
        self.level = 0

    # create  new node
    def createNode(self, lvl, key):
        n = Node(key, lvl)
        return n

    # create random level for node
    def randomLevel(self):
        lvl = 0
        while random.random() < self.P and \
                lvl < self.MAXLVL: lvl += 1
        return lvl

    # insert given key in skip list
    def insertElement(self, key):
        # create update array and initialize it
        update = [None] * (self.MAXLVL + 1)
        current = self.header

        '''
        start from highest level of skip list
        move the current reference forward while key 
        is greater than key of node next to current
        Otherwise inserted current in update and 
        move one level down and continue search
        '''
        for i in range(self.level, -1, -1):
            while current.forward[i] and \
                    current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

        ''' 
        reached level 0 and forward reference to 
        right, which is desired position to 
        insert key.
        '''
        current = current.forward[0]

        '''
        if current is NULL that means we have reached
           to end of the level or current's key is not equal
           to key to insert that means we have to insert
           node between update[0] and current node
       '''
        if current == None or current.key != key:
            # Generate a random level for node
            rlevel = self.randomLevel()

            '''
            If random level is greater than list's current
            level (node with highest level inserted in 
            list so far), initialize update value with reference
            to header for further use
            '''
            if rlevel > self.level:
                for i in range(self.level + 1, rlevel + 1):
                    update[i] = self.header
                self.level = rlevel

            # create new node with random level generated
            n = self.createNode(rlevel, key)

            # insert node by rearranging references
            for i in range(rlevel + 1):
                n.forward[i] = update[i].forward[i]
                update[i].forward[i] = n

            print("Successfully inserted key {}".format(key))

    def deleteElement(self, search_key):

        # create update array and initialize it
        update = [None] * (self.MAXLVL + 1)
        current = self.header

        '''
        start from highest level of skip list
        move the current reference forward while key 
        is greater than key of node next to current
        Otherwise inserted current in update and 
        move one level down and continue search
        '''
        for i in range(self.level, -1, -1):
            while (current.forward[i] and current.forward[i].key < search_key):
                current = current.forward[i]
            update[i] = current

        ''' 
        reached level 0 and advance reference to 
        right, which is possibly our desired node
        '''
        current = current.forward[0]

        # If current node is target node
        if current != None and current.key == search_key:

            '''
            start from lowest level and rearrange references 
            just like we do in singly linked list
            to remove target node
            '''
            for i in range(self.level + 1):

                '''
                If at level i, next node is not target 
                node, break the loop, no need to move 
                further level
                '''
                if update[i].forward[i] != current:
                    break
                update[i].forward[i] = current.forward[i]

            # Remove levels having no elements
            while (self.level > 0 and self.header.forward[self.level] == None):
                self.level -= 1
            print("Successfully deleted {}".format(search_key))

    def searchElement(self, key):
        current = self.header

        for i in range(self.level, -1, -1):
            while (current.forward[i] and current.forward[i].key < key):
                current = current.forward[i]

        # reached level 0 and advance reference to
        # right, which is prssibly our desired node
        current = current.forward[0]

        # If current node have key equal to
        # search key, we have found our target node
        if current and current.key == key:
            print("Found key ", key)
        else:
            print('Not Found')

    # Display skip list level wise
    def displayList(self):
        print("\n*****Skip List******")
        head = self.header
        for lvl in range(self.level + 1):
            print("Level {}: ".format(lvl), end=" ")
            node = head.forward[lvl]
            while (node != None):
                print(node.key, end=" ")
                node = node.forward[lvl]
            print("")
            
    def find(self, key):
        current = self.header

        for i in range(self.level, -1, -1):
            while (current.forward[i] and current.forward[i].key < key):
                current = current.forward[i]
        current = current.forward[0]

        if current and current.key == key:
            print("Found key ", key)
            return True
        else:
            return False

In [63]:
stock1

['17-Oct-2022',
 'EQ',
 '215.00',
 '218.50',
 '213.85',
 '215.90',
 '218.10',
 '217.95',
 '216.50',
 '298.05',
 '190.00',
 '94,20,098',
 '2,03,94,86,139.00',
 '69,021',
 '14-Oct-2022',
 'EQ',
 '220.00',
 '220.60',
 '215.50',
 '216.15',
 '216.30',
 '215.90',
 '217.97',
 '298.05',
 '177.00',
 '87,39,363',
 '1,90,49,59,296.05',
 '62,566',
 '13-Oct-2022',
 'EQ',
 '217.95',
 '218.90',
 '213.50',
 '217.95',
 '216.30',
 '216.15',
 '215.73',
 '298.05',
 '177.00',
 '95,59,072',
 '2,06,21,61,589.10',
 '68,244',
 '12-Oct-2022',
 'EQ',
 '218.00',
 '219.40',
 '215.05',
 '217.15',
 '217.90',
 '217.95',
 '217.30',
 '298.05',
 '177.00',
 '95,04,518',
 '2,06,53,73,231.85',
 '73,471',
 '11-Oct-2022',
 'EQ',
 '220.40',
 '222.25',
 '216.50',
 '220.35',
 '217.00',
 '217.15',
 '219.51',
 '298.05',
 '177.00',
 '90,06,207',
 '1,97,69,26,095.90',
 '76,479',
 '10-Oct-2022',
 'EQ',
 '220.00',
 '220.75',
 '217.10',
 '222.50',
 '220.55',
 '220.35',
 '219.10',
 '298.05',
 '177.00',
 '95,16,511',
 '2,08,50,49,462.50

In [64]:
# SkipList Class Takes Max_Levels And Probability As Input
skip_list_stock1 = SkipList(5, 0.5)
for i in range(5,len(stock1),14):
    skip_list_stock1.insertElement(float(stock1[i]))

Successfully inserted key 215.9
Successfully inserted key 216.15
Successfully inserted key 217.95
Successfully inserted key 217.15
Successfully inserted key 220.35
Successfully inserted key 222.5
Successfully inserted key 221.7
Successfully inserted key 219.05
Successfully inserted key 213.15
Successfully inserted key 216.5
Successfully inserted key 212.25
Successfully inserted key 213.9
Successfully inserted key 217.5
Successfully inserted key 215.3
Successfully inserted key 224.5
Successfully inserted key 231.65
Successfully inserted key 231.2
Successfully inserted key 235.25
Successfully inserted key 232.35
Successfully inserted key 236.3
Successfully inserted key 245.4
Successfully inserted key 241.5
Successfully inserted key 241.05
Successfully inserted key 242.8
Successfully inserted key 242.35
Successfully inserted key 244.9
Successfully inserted key 246.95
Successfully inserted key 247.35
Successfully inserted key 236.45
Successfully inserted key 235.2
Successfully inserted key

In [65]:
skip_list_stock1.displayList()


*****Skip List******
Level 0:  212.25 213.15 213.9 215.3 215.9 216.15 216.5 217.15 217.5 217.95 219.05 220.35 221.7 222.5 224.15 224.5 227.65 228.25 228.5 229.3 229.4 229.6 231.2 231.45 231.65 232.1 232.35 232.4 232.5 232.65 235.1 235.2 235.25 236.0 236.3 236.45 236.95 237.2 237.3 237.95 241.05 241.5 242.35 242.8 244.9 245.4 246.95 247.35 
Level 1:  212.25 213.9 215.3 216.15 216.5 220.35 222.5 224.15 224.5 227.65 228.5 229.4 229.6 231.45 231.65 232.4 232.5 235.1 236.95 237.2 241.05 242.35 244.9 245.4 
Level 2:  216.5 220.35 222.5 224.5 227.65 228.5 229.4 231.65 232.4 232.5 235.1 241.05 244.9 
Level 3:  216.5 220.35 222.5 229.4 235.1 
Level 4:  220.35 229.4 235.1 
Level 5:  229.4 


In [66]:
skip_list_stock2 = SkipList(5, 0.5)
for i in range(5,len(stock2),14):
    skip_list_stock2.insertElement(float(stock2[i]))

Successfully inserted key 331.25
Successfully inserted key 346.85
Successfully inserted key 356.8
Successfully inserted key 356.85
Successfully inserted key 365.35
Successfully inserted key 372.85
Successfully inserted key 370.1
Successfully inserted key 352.5
Successfully inserted key 354.85
Successfully inserted key 373.5
Successfully inserted key 369.45
Successfully inserted key 367.75
Successfully inserted key 372.55
Successfully inserted key 366.5
Successfully inserted key 385.6
Successfully inserted key 396.25
Successfully inserted key 390.5
Successfully inserted key 398.25
Successfully inserted key 391.85
Successfully inserted key 387.75
Successfully inserted key 399.2
Successfully inserted key 384.55
Successfully inserted key 393.6
Successfully inserted key 397.6
Successfully inserted key 397.0
Successfully inserted key 401.9
Successfully inserted key 406.3
Successfully inserted key 407.75
Successfully inserted key 389.85
Successfully inserted key 401.25
Successfully inserted k

In [67]:
skip_list_stock2.displayList()


*****Skip List******
Level 0:  103.45 108.75 111.2 112.2 117.05 123.2 331.25 346.85 352.5 354.85 356.8 356.85 365.35 366.5 367.75 369.45 370.1 372.55 372.85 373.5 375.5 384.55 385.6 387.75 389.85 390.4 390.5 391.85 393.5 393.6 394.25 396.25 397.0 397.6 398.25 399.2 401.25 401.9 406.3 407.75 408.5 413.95 
Level 1:  108.75 117.05 331.25 352.5 354.85 356.8 356.85 366.5 369.45 370.1 372.85 389.85 393.5 396.25 397.6 398.25 399.2 401.25 401.9 406.3 
Level 2:  108.75 117.05 331.25 352.5 356.8 366.5 370.1 393.5 396.25 397.6 401.25 401.9 
Level 3:  352.5 366.5 396.25 401.9 
Level 4:  352.5 
Level 5:  352.5 


In [98]:
def intersection(list1,list2):
    lst = []
    head  = list1.header
    temp = head.forward[0]
    while temp!=None:
        if(list2.find(temp.key)):
            lst.append(temp.key)
        temp = temp.forward[0]
    return lst

In [99]:
list1 = SkipList(4,0.5)
list1.insertElement(19)
list1.insertElement(20)
list1.insertElement(4)
list1.insertElement(5)
list1.insertElement(7)

Successfully inserted key 19
Successfully inserted key 20
Successfully inserted key 4
Successfully inserted key 5
Successfully inserted key 7


In [100]:
list2 = SkipList(4,0.5)
list2.insertElement(19)
list2.insertElement(20)
list2.insertElement(5)
list2.insertElement(8)
list2.insertElement(7)

Successfully inserted key 19
Successfully inserted key 20
Successfully inserted key 5
Successfully inserted key 8
Successfully inserted key 7


In [101]:
intersection(list1,list2)

Found key  5
Found key  7
Found key  19
Found key  20


[5, 7, 19, 20]

In [72]:
def min_element(list1):
    head = list1.header
    temp = head.forward[0]
    return temp.key

In [73]:
min_element(list1)

4

In [74]:
min_element(list2)

5

In [75]:
def max_element(list1):
    head = list1.header
    temp = head.forward[0]
    while temp.forward[0] != None:
        temp = temp.forward[0]
    return temp.key

In [76]:
max_element(list1)

20

In [77]:
max_element(list2)

20

In [78]:
list1.searchElement(2)

Not Found


In [79]:
list1.searchElement(5)

Found key  5


In [18]:
# PLOT VISUALIZATION